In [2]:
import numpy as np
from sklearn.datasets import make_classification
from torch import nn
import torch
from skorch import NeuralNetClassifier

Data import

In [3]:
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=10,
    random_state=87
)
X, y = np.array(X, dtype=np.float32), np.array(y, dtype=np.int64)

Create model

In [5]:
class Net(nn.Module):
    def __init__(self, num_units=10, nonlin=nn.ReLU()):
        super().__init__()
        
        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nonlin
        self.dense1 = nn.Linear(num_units, num_units)
        self.out = nn.Linear(num_units, 2)
        
    def forward(self, X):
        x = self.nonlin(self.dense0(X))
        x = self.dropout(x)
        x = self.nonlin(self.dense1(x))
        x = self.out(x)
        return x

Create classifier, train and evaluate model

In [9]:
net = NeuralNetClassifier(
    Net,
    max_epochs=15,
    lr=0.1,
    criterion=nn.CrossEntropyLoss,
    optimizer=torch.optim.SGD,
    iterator_train__shuffle=True
)

train_history = net.fit(X, y)

y_proba = net.predict_proba(X)
y_pred = net.predict(X)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6815       0.7000        0.6461  0.0478
      2        0.6280       0.7700        0.6059  0.0066
      3        0.5827       0.7950        0.5600  0.0086
      4        0.5369       0.8100        0.5229  0.0057
      5        0.4971       0.8150        0.4919  0.0074
      6        0.4655       0.8250        0.4720  0.0056
      7        0.4406       0.8250        0.4562  0.0078
      8        0.4191       0.8300        0.4373  0.0059
      9        0.4025       0.8350        0.4245  0.0068
     10        0.3882       0.8200        0.4154  0.0059
     11        0.3737       0.8150        0.4038  0.0061
     12        0.3630       0.8250        0.3956  0.0070
     13        0.3498       0.8450        0.3969  0.0067
     14        0.3389       0.8400        0.3887  0.0075
     15        0.3258       0.8500        0.3856  0.0062


sklearn pipeline

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

net = NeuralNetClassifier(
    Net,
    max_epochs=15,
    lr=0.1,
    criterion=nn.CrossEntropyLoss,
    optimizer=torch.optim.SGD,
    iterator_train__shuffle=True
)

pipe = Pipeline([
    ('scale', scaler),
    ('net', net)
])

train_history = pipe.fit(X, y)

y_proba = pipe.predict_proba(X)
y_pred = pipe.predict(X)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6939       0.5450        0.6872  0.0165
      2        0.6888       0.5900        0.6826  0.0101
      3        0.6839       0.6350        0.6776  0.0114
      4        0.6782       0.6400        0.6708  0.0093
      5        0.6704       0.6700        0.6612  0.0075
      6        0.6585       0.7000        0.6483  0.0093
      7        0.6418       0.7300        0.6297  0.0078
      8        0.6183       0.7550        0.6033  0.0097
      9        0.5852       0.7550        0.5723  0.0096
     10        0.5473       0.7750        0.5387  0.0069
     11        0.5071       0.7900        0.5066  0.0080
     12        0.4677       0.7800        0.4815  0.0065
     13        0.4316       0.7850        0.4584  0.0075
     14        0.3988       0.8000        0.4422  0.0063
     15        0.3695       0.8000        0.4349  0.0078


sklearn grid search

In [16]:
from sklearn.model_selection import GridSearchCV

net = NeuralNetClassifier(
    Net,
    max_epochs=30,
    lr=0.1,
    criterion=nn.CrossEntropyLoss,
    optimizer=torch.optim.SGD,
    iterator_train__shuffle=True
)
# turn off train-test-split and verbose
net.set_params(train_split=False,
               verbose=0)

# parameters for grid search
param = {
    'lr': [0.01, 0.02, 0.05],
    'max_epochs': [10, 20, 30],
    'module__num_units': [10, 20, 30]
}

# perform grid search
gs = GridSearchCV(net, param_grid=param,
                  n_jobs=-1, refit=False,
                  cv=3, scoring='accuracy')
gs.fit(X, y)
print(gs.best_score_, gs.best_params_)


/opt/miniconda3/envs/pytorch/lib/python3.12/site-packages/skorch/net.py:2231: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cuda_attrs = torch.load(f, **load_kwargs)
/opt/mi

0.893995792199385 {'lr': 0.05, 'max_epochs': 30, 'module__num_units': 20}
